In [1]:
from collections import OrderedDict


# A set of descriptors for various types
class Typed:
    _expected_type = type(None) 
    
    def __init__(self, name=None):
        self._name = name
        
    def __set__(self, instance, value):
        print(f'instance {instance}')
        print(f'value {value}\n')
        
        if not isinstance(value, self._expected_type):
            raise TypeError('Expected ' + str(self._expected_type)) 
        instance.__dict__[self._name] = value
        
class Integer(Typed):
    _expected_type = int 
    
class Float(Typed):
    _expected_type = float 

class String(Typed):
    _expected_type = str

    # Metaclass that uses an OrderedDict for class body
class OrderedMeta(type):
    def __new__(cls, clsname, bases, clsdict):  # 以该类为元类的类的所有属性都在clsdict中，包括函数，类属性。
        d = dict(clsdict)
        order = []
        for name, value in clsdict.items():
            print(f'name {name}')
            print(f'value1 {value}\n')
            if isinstance(value, Typed):  # 将init中的几个参数判断下，然后存入列表中，之后以_order为key存入clsdict中
                value._name = name 
                order.append(name)
            d['_order'] = order
        return type.__new__(cls, clsname, bases, d)
        
    @classmethod
    def __prepare__(cls, clsname, bases):   # clsname属性的字典
#         print('cls: ', cls)  # OrderedMeta
#         print('clsname: ', clsname)  # 父类名（以该类为元类的类）
#         print('bases: ', bases)     # 基类
        return OrderedDict()  # 使得clsdict变成有序字典
    
class Structure(metaclass=OrderedMeta): 
    def as_csv(self):
        return ','.join(str(getattr(self,name)) for name in self._order)

    # Example use
class Stock(Structure): 
    name = String()
    shares = Integer()
    price = Float()
    
    def __init__(self, name, shares, price): 
        self.name = name
        self.shares = shares
        self.price = price

name __module__
value1 __main__

name __qualname__
value1 Structure

name as_csv
value1 <function Structure.as_csv at 0x108a7b730>

name __module__
value1 __main__

name __qualname__
value1 Stock

name name
value1 <__main__.String object at 0x108a906d8>

name shares
value1 <__main__.Integer object at 0x108a90940>

name price
value1 <__main__.Float object at 0x108a909e8>

name __init__
value1 <function Stock.__init__ at 0x108a7bea0>



In [2]:
s = Stock('GOOG',100, 490.1)
print(s.name)

instance <__main__.Stock object at 0x108a6fdd8>
value GOOG

instance <__main__.Stock object at 0x108a6fdd8>
value 100

instance <__main__.Stock object at 0x108a6fdd8>
value 490.1

GOOG


In [3]:
s.as_csv()

'GOOG,100,490.1'

In [4]:
t = Stock('AAPL','a lot', 610.23)
t

instance <__main__.Stock object at 0x108a6fda0>
value AAPL

instance <__main__.Stock object at 0x108a6fda0>
value a lot



TypeError: Expected <class 'int'>